# Imports

In [1]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, RYGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List
import numpy as np
from qiskit.opflow import X, Z, I, H
from qiskit.quantum_info import Pauli
import matplotlib
from qiskit import assemble, Aer
from qiskit import *
from qiskit.visualization import plot_histogram
import math
from qiskit.algorithms.optimizers import L_BFGS_B

# Implementing the ansatz from qiskit tutorial

In [2]:
def calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                qc.append(CXGate(), [qr[qubit_index], qr[qubit_index + 1]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_full_entangelment_ansatz(num_of_qubits, thetas, depth):
    if thetas.size != 2*num_of_qubits*(depth + 1):
        print("ERROR with thetas size")
    else:
        qr = QuantumRegister(num_of_qubits, name="qubit")
        qc = QuantumCircuit(qr)
        
        for iteration in range(depth):
            for qubit_index in range(num_of_qubits):
                RY_theta_index = iteration*2*num_of_qubits + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
            for qubit_index in range(num_of_qubits - 1):
                for target_qubit_index in range(qubit_index + 1, num_of_qubits):
                    qc.append(CXGate(), [qr[qubit_index], qr[target_qubit_index]])
        
        for qubit_index in range(num_of_qubits):
                RY_theta_index = 2*num_of_qubits*depth + qubit_index
                RZ_theta_index = RY_theta_index + num_of_qubits
                qc.append(RYGate(thetas[RY_theta_index]), [qr[qubit_index]])
                qc.append(RZGate(thetas[RZ_theta_index]), [qr[qubit_index]])
        
        return qc

def calc_ansatz_circ(num_of_qubits, entangelment, thetas, depth):
    if entangelment == "linear":
        return calc_linear_entangelment_ansatz(num_of_qubits, thetas, depth)
    assert entangelment == "full", "entangelment has to be linear or full"
    return calc_full_entangelment_ansatz(num_of_qubits, thetas, depth)

# Generating the hamiltonians(for 4 qubits)

In [3]:
# for convinience let all coefficients ai and Jij be 0.5
H_transverse_ising = 0.5*((I^I^I^X) + (I^I^X^I) + (I^X^I^I) + (X^I^I^I) + \
                          (Z^Z^I^I) + (Z^I^Z^I) + (Z^I^I^Z) + (I^Z^Z^I) + \
                          (I^Z^I^Z) + (I^I^Z^Z))

# TODO: change this to a 4 qubits Hamiltonian
H2_molecule_Hamiltonian = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

def get_hamiltonian(hamiltonian_type):
    if hamiltonian_type == "Ising Model":
        return H_transverse_ising
    assert hamiltonian_type == "Molecular", "hamiltonian_type has to be Ising Model or Molecular"
    return H2_molecule_Hamiltonian

# Expectation Value

In [28]:
reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_strings

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    print(pauli_string)
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(QUBITS_NUM, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()

    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result() 
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)

# Calculating the target function

In [5]:
def target_function(thetas):
    ansatz = calc_ansatz_circ(QUBITS_NUM, Entangelment, thetas, Depth)
    
    qr = QuantumRegister(QUBITS_NUM, name="qubit")
    qc = QuantumCircuit(qr)
    qc.initialize(Init_state, qc.qubits)
    
    total_expectation_value = 0
    pauli_strings = transfrom_hamiltonian_into_pauli_string(Ham)
    for pauli_string in pauli_strings:
        qc_per_pauli_string = qc.compose(ansatz)
        qc_per_pauli_string = get_z_reduction_for_pauli_string(qc_per_pauli_string, pauli_string)
        probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc_per_pauli_string)
        
        total_expectation_value += get_expectation_value(pauli_string, probas_distribution)
    print("total expectation value:")
    print(total_expectation_value)
    return total_expectation_value
        

# Implementing the VQE algorithem

In [17]:
def VQE(optimazer, initial_thetas):
    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=initial_thetas)
    print(point)
    print("---point---")
    print(value)
    print("---value---")

    old_thetas_opt = point
    old_value = value

    point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=old_thetas_opt)
    print(point)
    print("---point---")
    print(value)
    print("---value---")
    new_thetas_opt = point
    new_value = value
    diff = abs(new_value - old_value)

    while diff > Stop_optimization_threshold:
        old_thetas_opt = new_thetas_opt
        old_value = new_value
        point, value, nfev = optimazer.optimize(QUBITS_NUM*2*(Depth + 1), target_function, initial_point=old_thetas_opt)
        print(point)
        print("---point---")
        print(value)
        print("---value---")
        new_thetas_opt = point
        new_value = value
        diff = abs(new_value - old_value)

    base_energy = value
    print("The VQE result for the minimal energy of the Hamiltonian given is:")
    print(base_energy)

# Setting the simulator and optimazer

In [18]:
bfgs_optimizer = L_BFGS_B(maxiter=2)

simulator_backend = Aer.get_backend('aer_simulator') 

# VQE with Transverse Ising Model Hamiltonian + LINEAR Entangelment Ansatz

In [19]:
#Backend number of shots
NUM_SHOTS = 1024

Ham = get_hamiltonian("Ising Model")
Entangelment = "linear"
QUBITS_NUM = 4
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)
    

total expectation value:
3.498046875
total expectation value:
3.513671875
total expectation value:
3.603515625
total expectation value:
3.453125
total expectation value:
3.490234375
total expectation value:
3.541015625
total expectation value:
3.537109375
total expectation value:
3.6171875
total expectation value:
3.53125
total expectation value:
3.611328125
total expectation value:
3.54296875
total expectation value:
3.55078125
total expectation value:
3.55078125
total expectation value:
3.50390625
total expectation value:
3.55078125
total expectation value:
3.599609375
total expectation value:
3.6015625
total expectation value:
3.505859375
total expectation value:
3.43359375
total expectation value:
3.484375
total expectation value:
3.44140625
total expectation value:
3.5625
total expectation value:
3.552734375
total expectation value:
3.548828125
total expectation value:
3.345703125
total expectation value:
3.404296875
total expectation value:
3.451171875
total expectation value:
3.

total expectation value:
3.546875
total expectation value:
3.5390625
total expectation value:
3.4609375
total expectation value:
3.568359375
total expectation value:
3.525390625
total expectation value:
3.603515625
total expectation value:
3.48828125
total expectation value:
3.51953125
total expectation value:
3.607421875
total expectation value:
3.72265625
total expectation value:
3.49609375
total expectation value:
3.51171875
total expectation value:
3.51171875
total expectation value:
3.396484375
total expectation value:
3.533203125
total expectation value:
3.50390625
total expectation value:
3.44921875
total expectation value:
3.609375
total expectation value:
3.7109375
total expectation value:
3.58984375
total expectation value:
3.505859375
total expectation value:
3.658203125
total expectation value:
3.52734375
total expectation value:
3.578125
total expectation value:
3.373046875
total expectation value:
3.5546875
total expectation value:
3.4765625
total expectation value:
3.419

total expectation value:
3.529296875
total expectation value:
3.54296875
total expectation value:
3.521484375
total expectation value:
3.611328125
total expectation value:
3.52734375
total expectation value:
3.634765625
total expectation value:
3.634765625
total expectation value:
3.462890625
total expectation value:
3.46484375
total expectation value:
3.509765625
total expectation value:
3.580078125
total expectation value:
3.599609375
total expectation value:
3.533203125
total expectation value:
3.443359375
total expectation value:
3.513671875
total expectation value:
3.5546875
total expectation value:
3.587890625
total expectation value:
3.5234375
total expectation value:
3.41796875
total expectation value:
3.65625
total expectation value:
3.52734375
total expectation value:
3.548828125
total expectation value:
3.576171875
total expectation value:
3.427734375
total expectation value:
3.47265625
total expectation value:
3.54296875
total expectation value:
3.52734375
total expectation

total expectation value:
3.53515625
total expectation value:
3.45703125
total expectation value:
3.669921875
total expectation value:
3.568359375
total expectation value:
3.416015625
total expectation value:
3.59765625
total expectation value:
3.638671875
total expectation value:
3.62890625
total expectation value:
3.56640625
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31.]
---point---
3.50390625
---value---
total expectation value:
3.470703125
total expectation value:
3.517578125
total expectation value:
3.537109375
total expectation value:
3.625
total expectation value:
3.548828125
total expectation value:
3.45703125
total expectation value:
3.5390625
total expectation value:
3.529296875
total expectation value:
3.599609375
total expectation value:
3.6171875
total expectation value:
3.525390625
total expectation value:
3.5078125
total expectation value:
3.591796875
total expectation value:
3.55078125
to

total expectation value:
3.5234375
total expectation value:
3.4609375
total expectation value:
3.693359375
total expectation value:
3.439453125
total expectation value:
3.578125
total expectation value:
3.5
total expectation value:
3.52734375
total expectation value:
3.4453125
total expectation value:
3.55859375
total expectation value:
3.482421875
total expectation value:
3.599609375
total expectation value:
3.62890625
total expectation value:
3.673828125
total expectation value:
3.591796875
total expectation value:
3.53125
total expectation value:
3.55078125
total expectation value:
3.525390625
total expectation value:
3.58984375
total expectation value:
3.48828125
total expectation value:
3.521484375
total expectation value:
3.53515625
total expectation value:
3.494140625
total expectation value:
3.59765625
total expectation value:
3.52734375
total expectation value:
3.46875
total expectation value:
3.611328125
total expectation value:
3.58203125
total expectation value:
3.52734375


total expectation value:
3.39453125
total expectation value:
3.57421875
total expectation value:
3.5078125
total expectation value:
3.39453125
total expectation value:
3.458984375
total expectation value:
3.49609375
total expectation value:
3.611328125
total expectation value:
3.595703125
total expectation value:
3.5390625
total expectation value:
3.51953125
total expectation value:
3.595703125
total expectation value:
3.5234375
total expectation value:
3.498046875
total expectation value:
3.578125
total expectation value:
3.62109375
total expectation value:
3.603515625
total expectation value:
3.572265625
total expectation value:
3.634765625
total expectation value:
3.525390625
total expectation value:
3.560546875
total expectation value:
3.466796875
total expectation value:
3.62109375
total expectation value:
3.408203125
total expectation value:
3.6796875
total expectation value:
3.478515625
total expectation value:
3.59765625
total expectation value:
3.501953125
total expectation va

total expectation value:
3.5703125
total expectation value:
3.5546875
total expectation value:
3.68359375
total expectation value:
3.41015625
total expectation value:
3.61328125
total expectation value:
3.361328125
total expectation value:
3.521484375
total expectation value:
3.46484375
total expectation value:
3.599609375
total expectation value:
3.443359375
total expectation value:
3.5234375
total expectation value:
3.482421875
total expectation value:
3.51953125
total expectation value:
3.568359375
total expectation value:
3.51171875
total expectation value:
3.6015625
total expectation value:
3.5078125
total expectation value:
3.583984375
total expectation value:
3.6953125
total expectation value:
3.619140625
total expectation value:
3.455078125
total expectation value:
3.6015625
total expectation value:
3.556640625
total expectation value:
3.453125
total expectation value:
3.572265625
total expectation value:
3.50390625
total expectation value:
3.490234375
total expectation value:


total expectation value:
3.48046875
total expectation value:
3.41796875
total expectation value:
3.59375
total expectation value:
3.66015625
total expectation value:
3.580078125
total expectation value:
3.486328125
total expectation value:
3.646484375
total expectation value:
3.564453125
total expectation value:
3.640625
total expectation value:
3.439453125
total expectation value:
3.51953125
total expectation value:
3.4296875
total expectation value:
3.583984375
total expectation value:
3.533203125
total expectation value:
3.607421875
total expectation value:
3.572265625
total expectation value:
3.513671875
total expectation value:
3.61328125
total expectation value:
3.662109375
total expectation value:
3.564453125
total expectation value:
3.564453125
total expectation value:
3.51171875
total expectation value:
3.611328125
total expectation value:
3.482421875
[-5.78939395e-09  9.99999992e-01  1.99999998e+00  2.99999999e+00
  4.00000000e+00  4.99999999e+00  5.99999999e+00  6.99999998e+

# VQE with Transverse Ising Model Hamiltonian + FULL Entangelment Ansatz

In [ ]:
#Backend number of shots
NUM_SHOTS = 1024

Ham = get_hamiltonian("Ising Model")
Entangelment = "full"
QUBITS_NUM = 4
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

# VQE with H2 Molecule Hamiltonian + LINEAR Entangelment Ansatz

In [29]:
#Backend number of shots
NUM_SHOTS = 1024

Ham = get_hamiltonian("Molecular")
Entangelment = "linear"
QUBITS_NUM = 2
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
0.25390625
total expectation value:
0.2890625
total expectation value:
0.205078125
total expectation value:
0.28515625
total expectation value:
0.2578125
total expectation value:
0.3359375
total expectation value:
0.166015625
total expectation value:
0.3046875
total expectation value:
0.40625
total expectation value:
0.3046875
total expectation value:
0.333984375
total expectation value:
0.318359375
total expectation value:
0.3125
total expectation value:
0.341796875
total expectation value:
0.298828125
total expectation value:
0.291015625
total expectation value:
0.287109375
total expectation value:
0.728515625
total expectation value:
0.73046875
total expectation value:
0.77734375
total expectation value:
0.857421875
total expectation value:
0.841796875
total expectation value:
0.767578125
total expectation value:
0.833984375
total expectation value:
0.689453125
total expectation value:
0.79296875
total expectation value:
0.7421875
total expectation value:
0.

total expectation value:
0.296875
total expectation value:
0.234375
total expectation value:
0.328125
total expectation value:
0.2890625
total expectation value:
0.34765625
total expectation value:
0.2734375
total expectation value:
0.263671875
total expectation value:
0.255859375
total expectation value:
0.341796875
total expectation value:
0.1875
total expectation value:
0.349609375
total expectation value:
0.298828125
total expectation value:
0.322265625
total expectation value:
0.357421875
total expectation value:
0.203125
total expectation value:
0.248046875
total expectation value:
0.365234375
total expectation value:
0.3125
total expectation value:
0.373046875
total expectation value:
0.2578125
total expectation value:
0.27734375
total expectation value:
0.408203125
total expectation value:
0.2421875
total expectation value:
0.3125
total expectation value:
0.279296875
total expectation value:
0.298828125
total expectation value:
0.330078125
total expectation value:
0.32421875
to

total expectation value:
0.275390625
total expectation value:
0.322265625
total expectation value:
0.337890625
total expectation value:
0.37890625
total expectation value:
0.208984375
total expectation value:
0.306640625
total expectation value:
0.279296875
total expectation value:
0.34375
total expectation value:
0.341796875
total expectation value:
0.376953125
total expectation value:
0.2734375
total expectation value:
0.296875
total expectation value:
0.28125
total expectation value:
0.34765625
total expectation value:
0.376953125
total expectation value:
0.3671875
total expectation value:
0.322265625
total expectation value:
0.26953125
total expectation value:
0.228515625
total expectation value:
0.275390625
total expectation value:
0.33203125
total expectation value:
0.224609375
total expectation value:
0.29296875
total expectation value:
0.275390625
total expectation value:
0.251953125
total expectation value:
0.345703125
total expectation value:
0.279296875
total expectation val

# VQE with H2 Molecule Hamiltonian + FULL Entangelment Ansatz

In [30]:
#Backend number of shots
NUM_SHOTS = 1024

Ham = get_hamiltonian("Molecular")
Entangelment = "full"
QUBITS_NUM = 2
Depth = 3
Init_state = np.zeros(2**QUBITS_NUM)
Init_state[0] = 1
Stop_optimization_threshold = 100
initial_thetas = np.arange(QUBITS_NUM*2*(Depth + 1))
optimazer = bfgs_optimizer
VQE(optimazer, initial_thetas)

total expectation value:
0.279296875
total expectation value:
0.318359375
total expectation value:
0.287109375
total expectation value:
0.330078125
total expectation value:
0.337890625
total expectation value:
0.3046875
total expectation value:
0.291015625
total expectation value:
0.232421875
total expectation value:
0.466796875
total expectation value:
0.28125
total expectation value:
0.3125
total expectation value:
0.328125
total expectation value:
0.32421875
total expectation value:
0.30859375
total expectation value:
0.259765625
total expectation value:
0.3515625
total expectation value:
0.333984375
total expectation value:
0.271484375
total expectation value:
0.283203125
total expectation value:
0.216796875
total expectation value:
0.21875
total expectation value:
0.193359375
total expectation value:
0.193359375
total expectation value:
0.240234375
total expectation value:
0.205078125
total expectation value:
0.19921875
total expectation value:
0.244140625
total expectation value:

total expectation value:
0.3046875
total expectation value:
0.296875
total expectation value:
0.34765625
total expectation value:
0.322265625
total expectation value:
0.27734375
total expectation value:
0.255859375
total expectation value:
0.2265625
total expectation value:
0.2578125
total expectation value:
0.302734375
total expectation value:
0.341796875
total expectation value:
0.345703125
total expectation value:
0.37890625
total expectation value:
0.25
total expectation value:
0.287109375
total expectation value:
0.271484375
total expectation value:
0.271484375
total expectation value:
0.35546875
total expectation value:
0.37890625
total expectation value:
0.234375
total expectation value:
0.3671875
total expectation value:
0.396484375
total expectation value:
0.306640625
total expectation value:
0.240234375
total expectation value:
0.322265625
total expectation value:
0.29296875
total expectation value:
0.37890625
total expectation value:
0.3046875
total expectation value:
0.3476

total expectation value:
0.599609375
total expectation value:
0.62109375
total expectation value:
0.59765625
total expectation value:
0.6015625
total expectation value:
0.66015625
total expectation value:
0.51953125
total expectation value:
0.6015625
total expectation value:
0.564453125
total expectation value:
0.56640625
total expectation value:
0.4453125
total expectation value:
0.330078125
total expectation value:
0.384765625
total expectation value:
0.412109375
total expectation value:
0.435546875
total expectation value:
0.37890625
total expectation value:
0.333984375
total expectation value:
0.365234375
total expectation value:
0.421875
total expectation value:
0.3359375
total expectation value:
0.447265625
total expectation value:
0.5
total expectation value:
0.50390625
total expectation value:
0.451171875
total expectation value:
0.435546875
total expectation value:
0.333984375
total expectation value:
0.404296875
total expectation value:
0.244140625
total expectation value:
0.

total expectation value:
0.3671875
total expectation value:
0.38671875
total expectation value:
0.314453125
total expectation value:
0.248046875
total expectation value:
0.376953125
total expectation value:
0.32421875
total expectation value:
0.31640625
total expectation value:
0.271484375
total expectation value:
0.3046875
total expectation value:
0.33203125
total expectation value:
0.3359375
total expectation value:
0.359375
total expectation value:
0.2578125
total expectation value:
0.330078125
total expectation value:
0.416015625
total expectation value:
0.2890625
total expectation value:
0.380859375
total expectation value:
0.28515625
total expectation value:
0.349609375
total expectation value:
0.357421875
total expectation value:
0.33984375
total expectation value:
0.25
total expectation value:
0.341796875
total expectation value:
0.23828125
total expectation value:
0.30859375
total expectation value:
0.25
total expectation value:
0.361328125
total expectation value:
0.359375
to